In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('./APCspend2013.csv', encoding = 'latin-1')

In [ ]:
df.head()

*On the basis of the information on the readme file and our visual observation of this data,For a its proper usage and manipulation and also precision of our statistical data analysis,we have some cleaning work to do.*

*Thus in order to determine the five most common journals and the total articles for each them,the number one
challenge is seeing the same journal writen different ways.Some of them are abbreviated and some of them are shortened which is difficult to run it as is as it will treat the same journals as different and this inturn messes our analysis.eg. 'J Med Chem' is the same as 'Journal of Medicinal Chemistry' but treated as different category.We also do not have information as wheather they are really the same or they just look the same.so to clean all it requires conciderable amount of time and manual work.*


lets remove the null values from our main study variable.

In [ ]:
Non_null_df = df.dropna(subset=['Journal title'])

In [ ]:
import re
Journal_title = Non_null_df['Journal title']
Journal_title.head()

From many of the values in the 'Journal title' field,we can read a common pattern that the same Journals' names are
written differently but abbrevated to mean the same journal.So based on that pattern lets just take the first letter 
of each word in the journal title name and that way they have the same values.

In [ ]:
segment_jtitle = Journal_title.apply(lambda x: re.findall(r'[A-Z][a-z]*', x))
segmented_df = pd.DataFrame(segment_jtitle)
segmented_df.head()
segmented_df['Abrevated_JT'] = segmented_df['Journal title'].apply(lambda x: ''.join(map(lambda y: y[0] , x)))
new_df = Non_null_df.merge(segmented_df,right_index=True, left_index = True)
new_df.head()

we want to use the Cost field values for statistical quantitative data analysis.there fore we want convert the values 
to float getting ride of the currency sign.

In [ ]:
new_df['Cost'] = new_df['COST (£) charged to Wellcome (inc VAT when charged)'].str.split('£').str[1]
new_df['Cost'] = new_df['Cost'].apply(lambda x: float(x))

In [ ]:
new_df.head()

*To complete this challenge, determine the five most common journals and the total articles for each?
the number one question in mind for answering this question is,what determines the commonity of journals.Are they common based on their cost or number of articles in each journal?or there is a secret code in the PMID/PMCID?In this case lets try to determine that based on the number of articles in each journal.*

In [ ]:
new_df['Abrevated_JT'].value_counts().head()

In [ ]:
new_df[['Abrevated_JT','Journal title_x']][new_df['Abrevated_JT'] == 'PLSO'].head(2)

In [ ]:
new_df[['Abrevated_JT','Journal title_x']][new_df['Abrevated_JT'] == 'PLSONE'].head(2)

from the two above senarios we can learn that we still have more cleaning work to do.The two Journal titles are 
spelled in different capslog.so lets try to fix that at the top of the equation before we abrevate it.

In [ ]:
jr_title = Journal_title.apply(lambda f: ' '.join(map(lambda g: g.capitalize(),f.split())))
jr_title.head()

In [ ]:
segment_jrtitle = jr_title.apply(lambda x: re.findall(r'[A-Z][a-z]*', x))
segmented_df1 = pd.DataFrame(segment_jrtitle)
segmented_df1['Abrevated_JT'] = segmented_df1['Journal title'].apply(lambda x: ''.join(map(lambda y: y[0] , x)))
new_df1 = Non_null_df.merge(segmented_df1,right_index=True, left_index = True)
new_df1.head()

Lets run again the above code to see if we have achieved our goal of cleaning

We can see the PO now is combined from several the same values like the once we saw in the above.

In [ ]:
new_df1['Abrevated_JT'].value_counts().head()

Therefore the above five journals are are the most common ones based on the number of articles published in the 
journals.

In [ ]:
five_journals = new_df1[new_df1['Abrevated_JT'].isin(['PO','N','B','JOBC','P'])]
five_journals['Cost'] = five_journals['COST (£) charged to Wellcome (inc VAT when charged)'].str.split('£').str[1]
five_journals['Cost'] = five_journals['Cost'].apply(lambda x: float(x))
five_journals.head()

*Therefore the following are the descriptive statistics for the open-access cost per article for each journal for the most common five journals*

In [ ]:
five_journals.groupby('Abrevated_JT').describe()

From the above description,we can learn that the data points are extremely diverse and the standard deviation for 
three Journals out of the five is way larger than the mean,therefore in this case median is more accurate measure 
of central tendency.

In [ ]:
plt.figure(figsize = (12,5))
plt.subplot(1,2,1)
sns.swarmplot(x = 'Abrevated_JT',y = 'Cost', data = five_journals)
plt.title('Swarm plot Journals and Cost', fontweight = 'bold')
plt.subplot(1,2,2)
sns.boxplot(x = 'Abrevated_JT',y = 'Cost', data = five_journals)
plt.title('Boxplot plot Journals and Cost', fontweight = 'bold')
plt.tight_layout()